# Quadratic binary optimization for problems with inequality constraints
Based on the [Constrained Quadratic Model](https://docs.ocean.dwavesys.com/en/stable/concepts/cqm.html) - CQM.

# 1 Theory
The `cost function` $C_f(\boldsymbol{x})$ can be represented as
\begin{equation}
C_f(\boldsymbol{x})=\sum_{i} c_{ii} x_i + \sum_{i < j} c_{ij} x_i x_j + const,
\label{eq:costcqm}
\end{equation} where $c_{ii}$ and $c_{ij}$ are real coefficients, and the variables $\{ x_i\}_{i=1, \dots, N}$ are binary.
---
<br> Similarly, the `constraints` are given by functions $G^{(k)}_{f}(\boldsymbol{x})$ can be represented as 
\begin{equation}
 G^{(k)}_{f}(\boldsymbol{x})=\sum_i g^{(k)}_{ii} x_i + \sum_{i < j} g^{(k)}_{ij} x_i x_j+ const \circ 0, 
 \label{eq:conscqm}
 \end{equation}
    where $g_{ii}$ and $g_{ij}$ are real coefficients, $k=1, \dots, M$ and $M$ is the number of constraints, and the symbol $\circ$ denotes a comparison operator  $\{\ge, \le, =\}.$
---    

<br> To use QAOA, all functions have to be combined into Quantum Unconstrained Binary Optimization. In the first step, this is done by transforming the constraints $G^{(k)}_{f}(\boldsymbol{x}) \circ 0 $ into the equality constraints $K^{(k)}_{f}(\boldsymbol{x})=0$. This is done using slack variables. 

---  
<br> Next, by adding weighted constraints and the cost function together, the `objective function` is obtained in the form
\begin{equation}
\label{qubo}
    f_{QUBO}(\boldsymbol{x})=\alpha_0C_f(\boldsymbol{x})+\sum_{k=1}^{M} {\alpha_{k} K_f^{(k)}(\boldsymbol{x})},
\end{equation}
where $\alpha_0$ is the weight of the cost function and $\alpha_{k}$ is the weight of the $k$-th constraint, $\alpha_i > 0$.

# 2 Simplified example
_Minimize $2x_0 + 5x_1 + x_0x_1$ subject to $x_0 + x_1 = 1 $ and $5x_0 + 2x_1 \leq 5 $_

Using the notation introduced in [1]
<center>
minimize $ C_f(\boldsymbol{x}) = 2x_0 + 5x_1 + x_0x_1$
    <br> subject to $G_f^{(1)}(\boldsymbol{x}) = x_0 + x_1 - 1 = 0$ and $G_f^{(2)}(\boldsymbol{x})=5x_0 + 2x_1 - 5 \leq 0 $

We need to convert the inequality constraint $G_f^{(2)}$ into the equality constraint $K_f^{(2)}$ using slack variables. To see how this is done, see the method `calc_slack_coefficients`. In our case $5$ can be expressed as $2^0 + 2^1 + 2$, so the coefficients for binary variables $\textbf{s}$ are $[1, 2, 2]$. In general, if the number is not a power of 2, the last coefficient is the difference between the previous power of 2 that falls within this number and this number.

<br>
<center>
$K_f^{(1)}(\boldsymbol{x}) = x_0 + x_1 - 1$ and $K_f^{(2)}(\boldsymbol{x})=5x_0 + 2x_1 - 1x_2 - 2x_3 -  2x_4 = 0 $

In QHyper, this problem needs to be a subclass of the [Problem class](https://github.com/qc-lab/QHyper/blob/tmek/improve-tutorials/QHyper/problems/base.py).

In [2]:
import numpy as np
import sympy
from sympy.core.expr import Expr
import sys
sys.path.append('..')
from QHyper.problems.base import Problem
from QHyper.util import Expression

class SimpleProblem(Problem):
    
    def __init__(self) -> None:
        self.slack_coefficients = calc_slack_coefficients(5)
        self.variables = sympy.symbols(' '.join([f'x{i}' for i in range(2)])  + ' ' 
                                    + ' '.join([f'x{i+2}' for i in range(len(self.slack_coefficients))]))
        self._set_objective_function()
        self._set_constraints()
        
    def _set_objective_function(self) -> None:
        C_f = 2 * self.variables[0] + 5 * self.variables[1] + self.variables[0] * self.variables[1]
        self.objective_function = Expression(C_f)
        
    def _set_constraints(self):
        K_f1 = self.variables[0] + self.variables[1] - 1
        
        K_f2 = 5 * self.variables[0] + 2 * self.variables[1] 
        for i, coefficient in enumerate(self.slack_coefficients):
           K_f2 += - coefficient * self.variables[i+2]
        #self.constraints = [Expression(K_f1)]    
        self.constraints = [Expression(K_f1), Expression(K_f2)]
    
    #########################
    def get_score(self, result, penalty=0):
        # "10000"
        # this function is used to evaluate the quality of the result
        
        x = [int(val) for val in result]
       
        if x[0] + x[1] -1 == 0 and 5 * x[0] + 2 * x[1] <= 5  and 5*x[0] + 2*x[1] - x[2] - 2*x[3] - 2*x[4] == 0:
           # print(x,  2 * x[0] + 5 * x[1]+ x[0] * x[1])
            return 2 * x[0] + 5 * x[1]+ x[0] * x[1]
       # print(x,penalty)
        return penalty
    #########################

In [3]:
def calc_slack_coefficients(constant: int) -> list[int]:
    num_slack = int(np.floor(np.log2(constant)))
    slack_coefficients = [2 ** j for j in range(num_slack)]
    if constant - 2 ** num_slack >= 0:
        slack_coefficients.append(constant - 2 ** num_slack + 1)
    return slack_coefficients

In [4]:
calc_slack_coefficients(5)

[1, 2, 2]

In [5]:
problem = SimpleProblem()

In [6]:
print(f"Variables used to describe objective function"
      f" and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints (RHS == 0):")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective function and constraints: (x0, x1, x2, x3, x4)
Objective function: {('x0', 'x1'): 1, ('x0',): 2, ('x1',): 5}
Constraints (RHS == 0):
    {('x0',): 1, ('x1',): 1, (): -1}
    {('x0',): 5, ('x1',): 2, ('x2',): -1, ('x3',): -2, ('x4',): -2}


# 3 Using QHyper

In [7]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function
from QHyper.solvers.vqa.base import VQA
tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(problem, config=tester_config)

In [8]:
# Create a VQA instance with QAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
   'pqc': {
       'type': 'qaoa',
       'layers': 5,
       'mixer': 'pl_x_mixer',
    },
   'optimizer': {
      'type': 'scipy',
      'maxfun': 200,
    },
}
vqa = VQA(problem, config=solver_config)

In [9]:
from QHyper.solvers import Solver
hyper_optimizer_bounds = [(1, 3), (1,1.1),(1,3)]
solver_config2 = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "qaoa",
                    "layers": 5,
                }
            }
        }
    },
    "hyper_optimizer": {
        "type": "random",    
        "processes": 1,
        "number_of_samples": 1,
        "bounds": hyper_optimizer_bounds
    }
}
vqa2 = Solver.from_config(problem, solver_config2)

In [14]:
# 2. Providing actual isntance of each class like VQA and Optimizer
# from QHyper.solvers.vqa.pqc.qaoa import QAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, QAOA(layers=5, mixer='pl_x_mixer'), ScipyOptimizer(maxfun=10))

Initialization part
<br> 'angles' -  QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
<br> 'hyper_args': [1, 1, 1], #  those are the alpha values from Section 1 - Theory

In [10]:
params_config = {
        'angles': [[0.5]*5, [0.7]*5], # QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
       # 'hyper_args': [1, 1, 1], #  those are the alpha values from [1]
       'hyper_args': [1, 1, 1]
    }

In [11]:
best_params = vqa2.solve(params_config)
print(f"Best params: {best_params}")

  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
best_results = tester.evaluate(best_params,  print_results=True)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyperoptimizer: {best_params['hyper_args']}")

NameError: name 'tester' is not defined

### Versions of the QAOA

1. 'Vanila' QAOA
<img src="imgs/qaoa.png" width="200" height="100">



'pqc': {
        'type': 'qaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'backend': 'default.qubit',
        },

2. 'Weight-free' WF-QAOA
<img src="imgs/wf-qaoa.png" width="200" height="100">



'pqc': {
        'type': 'wfqaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'penalty': 0.0
        'backend': 'default.qubit',
        },

3. 'Hyper' H-QAOA
<img src="imgs/hqaoa.png" width="200" height="100">



'pqc': {
        'type': 'hqaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'backend': 'default.qubit',
        },

### Versions of optimizers

'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
        'bounds': []
    },